# AD Utilisateur CC2 28 février 2020

## Données

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

dataf = pd.read_excel("Data/phones1.xls")
Xf = dataf._get_numeric_data()
n = Xf.shape[0]
colval = dataf.columns.values
dataf

In [ ]:
statistics = pd.DataFrame({'Feature':Xf.columns.values,
                           '$\overline{x}$': np.round(np.array(dataf.mean()),2),
                           '$s^2$': np.round(np.array(dataf.var(ddof=0)),2)})   
statistics

# Exercice 1

## 1) Les 2 premiers smartphones regroupés sont \#2 et \#7.  Calculez quelle devrait être la valeur de l'indice correspondant pour qu'il s'agisse bien de  l'accroissement d'inertie intra-groupes. 

In [ ]:
x2 = Xf.iloc[1]
x7 = Xf.iloc[6]
Dw = np.dot(x2-x7,x2-x7) / 2 / n
Dw

## 2) Quelle partition retiendriez-vous, et pourquoi~?

## Le plus grand accroissement d'inertie intra-groupes suggère de choisir une partition en 3 clusters (coupe en 40).

## 3) Complétez le canevas ci-après dont le but sera de réaliser ce clustering, puis de comparer la partition résultante à la variable catégorielle Supply.

## [File : smartphones] -> [Distances] -> [Hierarchical Clustering] -> [Sieve Diagram]

# Exercice 2

In [ ]:
X = Xf[['Bell','Auto.','Use']].values
Xt = X.T

In [ ]:
x = np.array([60, 50, 75])
y = np.array([17,70,70])
z = np.array([18,80,65])

In [ ]:
dinf = pd.DataFrame({'Ident.':dataf['Ident.'],
                  'x':np.array([39., 41., 40., 46., 40., 43., 43., 40.]),
                  'y':np.array([10., 30., 2., 26., 10., 24., 30., 11.]),
                  'z':np.array([10., 30., 3., 16., 9., 23., 30., 5.])
                 })   
dinf

## 1) Calculez la distance manquante.

In [ ]:
x7 = X[6]
max(abs(x-x7))

## 2) Classez $x$, $y$ et $z$ (3-NN, $d_\infty$)

In [ ]:
labels = np.unique(dataf['Supply'])
votes = np.zeros(len(labels))
probas = np.zeros(len(labels))

## Il faudrait tirer au sort certaines valeurs ; ci-dessous, c'est en python...
## Remplacez 'z' par 'x' et 'y' pour les autres classements (prédictions).

## x : (#1,#3,#5), [3, 0, 0], [1. 0. 0.], AA
## y : (#3,#1,#5), [3, 0, 0], [1. 0. 0.], AA
## z : (#3,#8,#5), [2, 1, 0], [0.66666667 0.33333333 0], AA

In [ ]:
K = 3
KNN = dinf['z'].argsort()[:K]
KNNid = dataf['Ident.'][KNN]
KNNlabels = dataf['Supply'][KNN]
for k in range(len(labels)):
    votes[k] = sum(KNNlabels==labels[k])
    probas[k] = votes[k] / K
print("voisins :\n",dataf['Ident.'][KNN])
print("votes :",votes)
print("probas :",probas)
prediction = labels[probas.argmax()]
print("prédiction :",prediction)

# Exercice 4

## 1) Faites-les calculs permettant de donner les valeurs manquantes dans les tableaux.

In [ ]:
bauf = dataf[['Bell','Auto.','Use']].copy()
X = bauf._get_numeric_data()
Y0 = [3,2,1,1,2,3,3,2]
bauf['Y'] = Y0
V1 = bauf.groupby("Y").mean()
V1

In [ ]:
from scipy.spatial.distance import pdist, cdist
D2 = np.round((cdist(X,V1)**2).T,2)
D2

In [ ]:
Y1 = D2.argmin(axis=0) + 1
bauf['Y'] = Y1
bauf

In [ ]:
V2 = bauf.groupby("Y").mean()
V2

In [ ]:
D2 = np.round((cdist(X,V2)**2).T,2)
D2

In [ ]:
Y2 = D2.argmin(axis=0) + 1
Y2

## 2) Calculez le plus simplement possible l'inertie intra-groupes de la partition $Y^{(2)}$.

In [ ]:
D2min = D2.min(axis=0)
D2min.mean()

## 3) L'inertie intra-groupes de la partition selon le constructeur vaut 66.26 ; cette partition est-elle meilleure que $Y^{(2)}$ ?

## Non puisque l'inertie intra-groupes est plus grande

## 4) Calculez la distance cosinus entre Bell et Auto. 

In [ ]:
Bell = bauf['Bell']
Bellp = Bell - Bell.mean()
Auto = bauf['Auto.']
Autop = Auto - Auto.mean()
#print("variables centrées :\n", Bellp, Autop)

BellpAutop = Autop.dot(Bellp)
BellpBellp = Bellp.dot(Bellp)
AutopAutop = Autop.dot(Autop)
print("produit scalaire, normes au carré :", BellpAutop, BellpBellp, AutopAutop)

dcos = 1 - BellpAutop / np.sqrt(BellpBellp*AutopAutop)
print("dcos :", dcos)

# Exercice 5

## 1) Pourrait/devrait-on regarder une troisième variable discriminante LD-z ? 

## On ne pourrait pas car il n'y en a que min(c-1=2,p=3) = 2

## 2) Sa projection sur LD-x vaut -27.7 ; calculez celle sur LD-y. 

In [ ]:
x = np.array([20,60,50,70,80])
LDy = np.array([0.95,0.00,-0.19,-0.09,-0.21])
x.dot(LDy)

## 3) On donne $d_1(x,\overline{x}_{AA}) = 11.3$ et $d_1(x,\overline{x}_{BB}) = 4.3$. Faites le reste des calculs nécessaires prédire quel fabricant (Supply) associer à ce nouveau smartphone. N'oubliez pas de conclure. 

In [ ]:
xproj = np.array([-27.7,-13.6])
xbarCC = np.array([-27.9,-17.3])
diff = (xproj-xbarCC)
diff

In [ ]:
(xproj-xbarCC).dot((xproj-xbarCC))

In [ ]:
13.73**0.5

## Avec la règle NP, on prédit le constructeur CC pour x

## 4) Visuellement, aurait-on la même prédiction avec la règle du Nearest Neighbor ? 

## En plaçant le point [-27.7, -13.6] sur la figure, il semble que son NN est soit le point #6 auquel cas la prédiction sera la même (CC), soit le point #8 auquel cas la prédiction sera BB